<a href="https://colab.research.google.com/github/Ayesha9811/Streamlit-Dataset-Creator-Sinhala/blob/main/MergedDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install datasets


In [21]:
from datasets import load_dataset, concatenate_datasets

source = load_dataset("Suchinthana/Sinhala-QA-Translate")
target = load_dataset("AyeshaKalpani98/Sinhala-QnA-Generate")


In [22]:
print(source['train'].column_names)
print(target['train'].column_names)


['Question', 'TranslatedQuestion', 'Answer', 'TranslatedAnswer']
['messages']


In [23]:
print(target['train'][0])

{'messages': [{'role': 'system', 'content': 'මගේ නම නවෝදි. මම ඔබට කෙසේද සහය වෙන්නේ?   '}, {'role': 'user', 'content': 'නාසා මූලස්ථානය පිහිටා ඇත්තේ කිනම් නගරයකද?'}, {'role': 'assistant', 'content': 'වොෂින්ටන් ඩී.සී'}]}


In [24]:
def transform_source(example):
    return {
        "messages": [
            {
                "role": "system",
                "content": "මගේ නම නවෝදි. මම ඔබට කෙසේද සහය වෙන්නේ?"
            },
            {
                "role": "user",
                "content": example["Question"]
            },
            {
                "role": "assistant",
                "content": example["TranslatedAnswer"]
            }
        ]
    }


In [25]:
transformed_source = source["train"].map(transform_source, remove_columns=["TranslatedQuestion", "Answer"])

In [26]:
transformed_source[0]

{'Question': 'කොවිඩ් ඩෙල්ටා ප්ලස් වයිරසය මුලින්ම වාර්තාවුන රට',
 'TranslatedAnswer': 'ඉන්දියාව',
 'messages': [{'content': 'මගේ නම නවෝදි. මම ඔබට කෙසේද සහය වෙන්නේ?',
   'role': 'system'},
  {'content': 'කොවිඩ් ඩෙල්ටා ප්ලස් වයිරසය මුලින්ම වාර්තාවුන රට',
   'role': 'user'},
  {'content': 'ඉන්දියාව', 'role': 'assistant'}]}

In [27]:
def fix_order(example):
    return {"messages": [{"role": m["role"], "content": m["content"]} for m in example["messages"]]}

target_aligned = target["train"].map(fix_order)

In [28]:
source_aligned = [fix_order(item) for item in transformed_source] # Iterate over the dataset rows, not 'messages' column

In [29]:
print(source_aligned[0])

{'messages': [{'role': 'system', 'content': 'මගේ නම නවෝදි. මම ඔබට කෙසේද සහය වෙන්නේ?'}, {'role': 'user', 'content': 'කොවිඩ් ඩෙල්ටා ප්ලස් වයිරසය මුලින්ම වාර්තාවුන රට'}, {'role': 'assistant', 'content': 'ඉන්දියාව'}]}


In [30]:
print(target_aligned[0])

{'messages': [{'role': 'system', 'content': 'මගේ නම නවෝදි. මම ඔබට කෙසේද සහය වෙන්නේ?   '}, {'role': 'user', 'content': 'නාසා මූලස්ථානය පිහිටා ඇත්තේ කිනම් නගරයකද?'}, {'role': 'assistant', 'content': 'වොෂින්ටන් ඩී.සී'}]}


In [31]:
from datasets import Dataset

source_aligned = Dataset.from_list(source_aligned)

In [32]:
def fix_message_order(example):
    example["messages"] = [{"role": msg["role"], "content": msg["content"]} for msg in example["messages"]]
    return example

# Ensure consistent field order for both datasets
source_aligned = source_aligned.map(fix_message_order)
target_aligned = target_aligned.map(fix_message_order)


Map:   0%|          | 0/1016 [00:00<?, ? examples/s]

In [33]:
def fix_message_order(example):
    return {
        "messages": [{"role": msg["role"], "content": msg["content"]} for msg in example["messages"]]
    }

source_fixed = source_aligned.map(fix_message_order, remove_columns=source_aligned.column_names)

Map:   0%|          | 0/1016 [00:00<?, ? examples/s]

In [34]:
from datasets import Dataset, Features, Sequence, Value

correct_schema = Features({
    "messages": Sequence({
        "role": Value("string"),
        "content": Value("string")
    })
})

source_fixed = Dataset.from_dict(source_fixed.to_dict(), features=correct_schema)

In [35]:
print(source_fixed.features)
print(target_aligned.features)

{'messages': Sequence(feature={'role': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}, length=-1, id=None)}
{'messages': [{'role': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}]}


In [36]:
def fix_message_order(example):
    return {
        "messages": [{"role": msg["role"], "content": msg["content"]} for msg in example["messages"]]
    }

source_fixed = source_aligned.map(fix_message_order, remove_columns=source_aligned.column_names)
target_fixed = target_aligned.map(fix_message_order, remove_columns=target_aligned.column_names)


Map:   0%|          | 0/1016 [00:00<?, ? examples/s]

In [37]:
from datasets import Dataset, Features, Sequence, Value

correct_schema = Features({
    "messages": Sequence({
        "role": Value("string"),
        "content": Value("string")
    })
})

# Convert dataset to dictionary and recreate with correct schema
source_fixed = Dataset.from_dict(source_fixed.to_dict(), features=correct_schema)
target_fixed = Dataset.from_dict(target_fixed.to_dict(), features=correct_schema)

print(source_fixed.features)
print(target_fixed.features)


{'messages': Sequence(feature={'role': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}, length=-1, id=None)}
{'messages': Sequence(feature={'role': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}, length=-1, id=None)}


In [38]:
from datasets import Dataset

source_fixed = Dataset.from_dict(source_fixed.to_dict(), features=correct_schema)
target_fixed = Dataset.from_dict(target_fixed.to_dict(), features=correct_schema)


In [39]:
merged_dataset = concatenate_datasets([target_fixed, source_fixed])

In [40]:
merged_dataset.select(range(5))

Dataset({
    features: ['messages'],
    num_rows: 5
})

In [41]:
len(merged_dataset)

1119

In [42]:
print(merged_dataset)
print(merged_dataset.features)

Dataset({
    features: ['messages'],
    num_rows: 1119
})
{'messages': Sequence(feature={'role': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}, length=-1, id=None)}


In [43]:
from huggingface_hub import login

login()  # Enter your Hugging Face token when prompted


In [44]:
merged_dataset.push_to_hub("AyeshaKalpani98/Sinhala-QnA-Generate")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AyeshaKalpani98/Sinhala-QnA-Generate/commit/9ce8b58e1ec37966470540b14b554d0a5862a570', commit_message='Upload dataset', commit_description='', oid='9ce8b58e1ec37966470540b14b554d0a5862a570', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/AyeshaKalpani98/Sinhala-QnA-Generate', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AyeshaKalpani98/Sinhala-QnA-Generate'), pr_revision=None, pr_num=None)